In [1]:
pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from scipy.stats import pearsonr

In [3]:
#loading the dataset
from datasets import load_dataset
import pandas as pd

# Load the dataset
dataset = load_dataset("nuvocare/WikiMedical_sentence_similarity", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/50712 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21735 [00:00<?, ? examples/s]

In [4]:
df = pd.DataFrame(dataset)


In [5]:
df

,text1,text2,label
0,Econometric studies have shown that this effec...,"""Spinraza access by country"". TreatSMA. 18 Oct...",0
1,"Recurrent silent strokes, with or without clin...","Tagraxofusp, sold under the brand name Elzonri...",0
2,Mutations in the ASS gene reduce the activity ...,Citrullinemia is an autosomal recessive urea c...,1
3,Fungal spores may be viewed directly on hair s...,"The lesions are located on the head, neck, tai...",1
4,The loss of function mutation results in a phe...,Common noninfectious tenosynovitis are: stenos...,0
...,...,...,...
50707,"Intermenstrual bleeding, previously known as m...","Sometimes, breakthrough bleeding is classified...",1
50708,The characteristic feature of arthritis is joi...,Toxic epidermal necrolysis (TEN) is a type of ...,0
50709,Signs and symptoms\nA manic episode is defined...,"In stages II and III mania, however, the patie...",1
50710,"The underdeveloped conjoined twin, Islaam, was...",Labial fusion is a medical condition of the fe...,0


In [6]:
label0 = df[df['label'] == 0].head(1000)
label1 = df[df['label'] == 1].head(1000)

In [7]:
train_dataset = pd.concat([label0, label1])

In [8]:
train_dataset = train_dataset.sample(frac=1).reset_index(drop=True)

In [9]:
train_dataset

,text1,text2,label
0,"Aniridia is a heterozygous disorder, meaning t...","Aniridia is the absence of the iris, a muscul...",1
1,Cancer\nCyclophosphamide is carcinogenic and m...,"However, in studies carried out after the clin...",1
2,"However, with almost universal use of ultrasou...",There are three types of vasa previa. Types 1 ...,1
3,The level of organisation varies: sometimes ga...,Drift\nthe slight lateral curved-path movement...,1
4,"With erythromycin treatment, individuals with ...",Lung abscess is a type of liquefactive necrosi...,0
...,...,...,...
1995,Although fibrothorax may not cause any symptom...,"Usually, the underlying condition has to cause...",1
1996,Angiogenic factors are crucial for vascular gr...,In November 2006 the American Dental Associati...,0
1997,"However, this method was never used widely, ev...",Iran also approved its own version of the mono...,0
1998,Activity: Many people with hepatitis prefer be...,"After conducting its investigation, the CDC at...",1


In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [11]:
#Preprocessing the data
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [12]:
train_dataset['text1'] = train_dataset['text1'].apply(preprocess_text)
train_dataset['text2'] = train_dataset['text2'].apply(preprocess_text)

In [13]:
train_dataset

,text1,text2,label
0,aniridia heterozygous disorder meaning one two...,aniridia absence iris muscular structure open ...,1
1,cancer cyclophosphamide carcinogenic may incre...,however study carried clinical efficacy cyclop...,1
2,however almost universal use ultrasound develo...,three type vas previa type 1 2 described catan...,1
3,level organisation varies sometimes game part ...,drift slight lateral curvedpath movement spinn...,1
4,erythromycin treatment individual dpb much lon...,lung abscess type liquefactive necrosis lung t...,0
...,...,...,...
1995,although fibrothorax may cause symptom people ...,usually underlying condition cause intense inf...,1
1996,angiogenic factor crucial vascular growth plac...,november 2006 american dental association publ...,0
1997,however method never used widely even soviet u...,iran also approved version monoclonal antibody...,0
1998,activity many people hepatitis prefer bed rest...,conducting investigation cdc attributed source...,1


In [14]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(train_dataset, test_size=0.2, random_state=42)


In [15]:
from tqdm import tqdm

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def evaluate_model(model, tokenizer, df, batch_size=8):
    num_samples = len(df)
    accuracy_list, precision_list, recall_list, f1_list, roc_list = [], [], [], [], []

    with tqdm(total=num_samples, desc="Evaluating") as pbar:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)

            inputs = tokenizer(
                df['text1'][start:end].tolist(),
                df['text2'][start:end].tolist(),
                return_tensors='pt',
                padding=True,
                truncation=True
            )
            labels = df['label'][start:end].values

            with torch.no_grad():
                outputs = model(**inputs)

            predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()

            # Update evaluation metrics for each batch
            accuracy = accuracy_score(labels, predictions)
            precision = precision_score(labels, predictions)
            recall = recall_score(labels, predictions)
            f1 = f1_score(labels, predictions)
            roc = roc_auc_score(labels, predictions)

            accuracy_list.append(accuracy)
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
            roc_list.append(roc)

            pbar.update(end - start)

    # Compute average metrics
    accuracy = sum(accuracy_list) / len(accuracy_list)
    precision = sum(precision_list) / len(precision_list)
    recall = sum(recall_list) / len(recall_list)
    f1 = sum(f1_list) / len(f1_list)
    mean_roc = sum(roc_list) / len(roc_list)

    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc': mean_roc
    }

    return results

# Define the models
models = {
    'roberta-base': 'roberta-base',
    'bert-base-uncased': 'bert-base-uncased',
    # 'paraphrase-MiniLM-L6-v2':'paraphrase-MiniLM-L6-v2',
    # 'msmarco-distilbert-base-v2': 'msmarco-distilbert-base-v2',
    'albert-base-v2': 'albert-base-v2',
    'distilbert-base-uncased': 'distilbert-base-uncased',
    'xlnet-base-cased': 'xlnet-base-cased',
}

# Store results for each model
results_dict = {}

# Evaluate each model
for model_name, model_key in models.items():
    # Load model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_key)
    tokenizer = AutoTokenizer.from_pretrained(model_key)

    # Evaluate the model
    results = evaluate_model(model, tokenizer, test_df)

    # Store results in the dictionary
    results_dict[model_name] = results

    # Print evaluation results
    print(f"{model_name}:")
    print(f"Accuracy: {results['accuracy']:.4f}, Precision: {results['precision']:.4f}, Recall: {results['recall']:.4f}, F1 Score: {results['f1']:.4f}")
    print(f"ROC: {results['roc']:.4f}\n")

# Access results_dict to get the evaluation results for each model
print(results_dict)


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Evaluating:  40%|████      | 160/400 [06:23<10:05,  2.52s/it]

In [ ]:
# Convert results_dict to a DataFrame
result_df = pd.DataFrame(results_dict).T

# Save DataFrame to a CSV file
result_df.to_csv('result.csv')

# Access results_df to get the evaluation results for each model
print(result_df)

In [ ]:
!pip install Topsis-Arpita-102103358

In [ ]:
!topsis result.csv "1,1,1,1,1" "+,+,+,+,+" final.csv

In [ ]:
ans=pd.read_csv('final.csv')

In [ ]:
ans

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


model_names = ans['Unnamed: 0']
topsis_scores = ans['Topsis Score']

# Plotting the bar graph
plt.figure(figsize=(6, 6))
plt.bar(model_names, topsis_scores, color='Red')
plt.xlabel('Model Name')
plt.ylabel('Topsis Score')
plt.title('Topsis Score for Different Models ')
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()